In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import time
%matplotlib inline

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import highway_conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization, batch_normalization
from tflearn.layers.estimator import regression

In [3]:
train = pd.read_csv("data/train.csv")

In [4]:
y_label = train["label"]

In [5]:
train = train.drop("label", axis=1)

In [6]:
train = np.array(train).reshape(32, 32, 3, -1).transpose(3,0,1,2)

In [7]:
y_label[y_label == 10] = 0
y_train = np.zeros([len(y_label), 10])
y_train[np.arange(len(y_label)), y_label] = 1

In [12]:
network = input_data(shape=[None, 32, 32, 3], name='input')
#highway convolutions with pooling and dropout
for i in range(3):
    for j in [3, 2, 1]: 
        network = highway_conv_2d(network, 16, j, activation='elu')
    network = max_pool_2d(network, 2)
    network = batch_normalization(network)
    
network = fully_connected(network, 128, activation='elu')
network = fully_connected(network, 256, activation='elu')
network = fully_connected(network, 10, activation='softmax')
model = tflearn.DNN(net, checkpoint_path='model_resnet_mnist',
                    max_checkpoints=10, tensorboard_verbose=0)
regress_layer = regression(network, optimizer='adam', 
                     loss='categorical_crossentropy',
                     learning_rate=1e-3)

TypeError: ones_initializer() got multiple values for argument 'dtype'

In [13]:
model = tflearn.DNN(regress_layer, checkpoint_path='model.tfl.ckpt', 
                    tensorboard_verbose=0, tensorboard_dir="tmp/logs", ='modelv0.2_best.ckpt')
model.fit(train, y_train, batch_size=128, n_epoch=20, validation_set=0.1, 
          shuffle=True, snapshot_step=10000, run_id="LeNet")

TypeError: __init__() got an unexpected keyword argument 'best_checkpoint_path'

In [ ]:
model.save(time.strftime(time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())))

In [ ]:
x_test = pd.read_csv("data/test.csv")

In [ ]:
x_test = np.array(x_test).reshape(32, 32, 3, -1).transpose(3,0,1,2)

In [ ]:
plt.imshow(x_test[3].astype("uint8"))

In [ ]:
res_list = list()
for i in x_test:
    pred = model.predict(i.reshape(-1,32,32,3))
    res_list.append(np.argmax(pred))

In [ ]:
res_list = np.array(res_list)
res_list[res_list == 0] = 10

res_output = pd.Dataframe({
    "ImageId": range(len(pred))
    "label": pred
})
filename = "res_" + time.strftime(time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())) + ".csv"
res_output.to_csv(filename, index=False)